In [1]:
import pandas as pd
from mpl_toolkits import mplot3d
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import argrelextrema
import os

In [2]:
def plot_colormap(data,title = "AUT and QUARTZ",path = ""):
    """
    Helper function to plot data with associated colormap.
    """
   # fig, axs = plt.subplots(1, n, figsize=(n * 2 + 2, 3),
    #                        constrained_layout=True, squeeze=False)
    figure, axes = plt.subplots(figsize=(data.shape[1]/5,data.shape[0]/5))
    psm = axes.pcolormesh(data, cmap='rainbow',rasterized=True)
    figure.colorbar(psm, ax=axes)
    axes.invert_yaxis()
    if abs(data.shape[1]-data.shape[0])<=1:
        print(data.shape[1],data.shape[0])
        axes.set_aspect('equal', adjustable='box')
    plt.title(title)
    #plt.savefig(path+" "+title)
    plt.show()

In [3]:
data_dir = "../data/221115_review/"
lst=os.listdir(data_dir)
files = []
for l in lst:
    if l[0] != '.':
        f_lst = os.listdir(data_dir+l)
        for f in f_lst:
            if '.png' not in f:
                files.append(data_dir+l+'/'+f)
files

['../data/221115_review/A_221015_250nm_negative/22o15003.abf-pqty=25_current.csv',
 '../data/221115_review/A_221015_250nm_negative/22o15003.abf_current_div-pqty=25.csv',
 '../data/221115_review/A_221015_250nm_negative/22o15003.abf_current_subs-pqty=25.csv',
 '../data/221115_review/A_221015_250nm_negative/22o15003.abf_EMF-pqty=25.csv',
 '../data/221115_review/A_221015_250nm_negative/22o15003.abf_EMF_close-pqty=25.csv',
 '../data/221115_review/A_221015_250nm_negative/22o15003.abf_EMF_div-pqty=25.csv',
 '../data/221115_review/A_221015_250nm_negative/22o15003.abf_EMF_far-pqty=25.csv',
 '../data/221115_review/A_221015_250nm_negative/22o15003.abf_EMF_subs-pqty=25.csv',
 '../data/221115_review/A_221015_250nm_positive/22o15002.abf-pqty=25_current.csv',
 '../data/221115_review/A_221015_250nm_positive/22o15002.abf_current_div-pqty=25.csv',
 '../data/221115_review/A_221015_250nm_positive/22o15002.abf_current_subs-pqty=25.csv',
 '../data/221115_review/A_221015_250nm_positive/22o15002.abf_EMF-pqty=

In [16]:
lst

['.ipynb_checkpoints',
 'A_221015_250nm_negative',
 'A_221015_250nm_positive',
 'B_221109_380nm_negative',
 'B_221109_380nm_positive',
 'C_221015_480nm_negative',
 'C_221015_480nm_positive',
 'D_221011_750nm_negative',
 'D_221011_750nm_postive']

In [ ]:
def process(path):
    if 'div' not in path and 'subs' not in path:
        return
    else:
        print(path)
    df = pd.read_csv(path,header = None)
    Z = np.transpose(np.array(df))
    
    # Plot 2 plots
    fig = plt.figure(figsize=(Z.shape[1]/2,Z.shape[0]/5))
    ax0 = plt.subplot2grid((2,3),(0,0))
    ax1 = plt.subplot2grid((2,3),(1,0))
    ax2 = plt.subplot2grid((2,3),(0,1),rowspan = 2,colspan = 2)
    ax0.invert_yaxis()
    ax1.invert_yaxis()
    ax0.set_aspect('equal', adjustable='box')
    ax1.set_aspect('equal', adjustable='box')
    
    psm = ax0.pcolormesh(Z, cmap='rainbow',rasterized=True)
    fig.colorbar(psm, ax=ax0)
    ax0.set_title("Original")
    
    bins = [(np.mean(Z))]
    z_dig = np.digitize(Z,bins)
    D = (1-z_dig)*Z
    '''
    if 'neg' in p and 'subs' in p:
        D = z_dig*Z
    else:
        D = (1-z_dig)*Z
    ''' 
    psm = ax1.pcolormesh(z_dig, cmap='cool',rasterized=True)
    fig.colorbar(psm, ax=ax1)
    ax1.set_title("After process")
    
    
    class1 = (z_dig*Z).flatten()
    class2 = ((1-z_dig)*Z).flatten()
    class1 = class1[class1 != 0]
    class2 = class2[class2 != 0]
    mean1 = np.mean(class1)
    std1 = np.std(class1)
    mean2 = np.mean(class2)
    std2 = np.std(class2)
    print(len(class1))
    ax2.hist(class1,bins=100,label = "{:.3f} +- {:.3f}".format(mean1,std1),color = 'aqua')
    ax2.hist(class2,bins=100,label = "{:.3f} +- {:.3f}".format(mean2,std2),color = 'magenta')
    ax2.legend(prop={'size': 30})

    plt.show()
    
    

In [ ]:
for p in files:
    process(p)